<a href="https://colab.research.google.com/github/WayneGretzky1/CSCI-4521-Applied-Machine-Learning/blob/main/2_4_PCA_on_iris_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Going back to the iris dataset, let's pick 3 features so we have 3D data and look at the results.

In [ ]:
import plotly.express as px

iris_data = px.data.iris() # this dataset is so common, its built into libraries!
features = [ "sepal_length", "petal_width", "petal_length"] #"sepal_width",

One way to visualize high-dimensional data is to plot pairs of features.

In [ ]:
fig = px.scatter_matrix(iris_data, dimensions=features, color="species")
fig.update_traces(diagonal_visible=False)
fig.show()

Where we are using the Plotly visualization library (an alternative to Matplotlib or Seaborn). Plotly lets you interact with its graphs! Hover your mouse over each point to get additional information.

Because we only have 3 features, we can also plot the data in 3D.

In [ ]:
fig = px.scatter_3d(iris_data, x = "petal_width", y="petal_length",  z = "sepal_length")
fig.show()

Again, Plotly lets you interact with the 3D plot!

Imagine you are going to take a 2D screenshot to include in a written report. Zoom in to the graph so that the points fill up the entire width or height of the plot as much as possible.

Then answer the following questions as today's in-class activity:

1.   While staying zoomed in, rotate the graph to find the worst possible angle! That is, find the angle that preserves the *least* amount of information about the true 3D structure of the data.
2.   Find the angle that preserves the *most* amount of information about the true 3D structure of the data.

*(upload both of these images as today's in-class activity)*

What makes a good rotation?


That's it, you now understand the key idea behind PCA. It's simply a matrix that rotates the data so you can still see as much of the original spread/variance as possible!


---

Let's try SKLearn's PCA package and see how it compares to your rotation.

Here is how you compute the PCA components for each data sample.

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
# TODO: perform PCA on iris data

pca = PCA()
components = pca.fit_transform(iris_data[features])
components_df = pd.DataFrame(data = components, columns = ["PC1", "PC2", "PC3"])
print(components_df.shape)
print(iris_data[features].shape)

(150, 3)
(150, 3)


We can see each sample now as a new 3D point:

In [ ]:
print(components_df)

          PC1       PC2       PC3
0   -2.656097  0.240044  0.026780
1   -2.728993  0.055838 -0.000680
2   -2.887767 -0.100825  0.015062
3   -2.752460 -0.248016 -0.085073
4   -2.692545  0.147941  0.013050
..        ...       ...       ...
145  1.946595  0.088737  0.476603
146  1.485011 -0.114438  0.151547
147  1.765671 -0.012857  0.181738
148  1.936108 -0.426867  0.321547
149  1.389087 -0.482857 -0.035711

[150 rows x 3 columns]


We can plot these points to see the PCA projection. How does it compare to your rotation?

In [ ]:
fig = px.scatter(components_df, x="PC1", y="PC2")
fig.show()

(Keep in mind, flipping the axis or rotating the graph around the centre all have the same "spread" - so there is a family of equally good PCA projections)

---

Here is the moment of truth! Our PCA projection never had access to the data labels. It's a completely unsupervised method. But, PCA captures some information about the structure of the underlying data by focusing us on the view of the data where there is the widest variety of features. This will often correlate with meaningful differences in the data, naturally separating different types of data from each other.

Let's view the PCA rotation, but this time color by label.

In [ ]:
fig = px.scatter(components_df, x="PC1", y="PC2", color = iris_data["species"])
fig.show()

Voilà! PCA left our datasets split into 3 pretty clean clusters *without ever knowing the labels*!


---
It's worth remembering that our principal components are just linear combinations of the features (chosen to be orthogonal and maximize variance). Nothing more!

To help emphasize that point, we can look at `pca.components_` to see the actual components of the linear transformation and print out the equations for each principal component.

In [ ]:
# Create a list of equations for each principal component
for i in range(len(pca.components_)):
  equation = ''
  for j in range(len(features)):
    if j > 0:
      equation += ' + '
    equation += str(round(pca.components_[i][j], 2)) + '*' + features[j]
  print('PC' + str(i+1) + ' = ' + equation)

PC1 = 0.36*sepal_length + 0.36*petal_width + 0.86*petal_length
PC2 = 0.92*sepal_length + -0.28*petal_width + -0.28*petal_length
PC3 = 0.14*sepal_length + 0.89*petal_width + -0.43*petal_length


---

So, how can we use PCA?

Firstly, PCA is a vital tool if you want to graph high-dimensional data. Even if you have a dataset with dozens or hundreds of features, using PCA to compute a 2D projection still often provides a meaningful way to visualize the dataset.

Secondly, working in the reduced dimension PCA space can be a good first step for other ML algorithms. For example, when implementing KNN, rather than measuring distances between two feature vectors directly, you can take the distance in PCA space. Often the first few PCA components contain most of the meaningful variation. In these cases, ignoring the less important PCA components removes noise from the dataset and focuses the predictor on what matters.

Let's use scikit-learn's built-in `train_test_split` function to build some training and testing data so we can see how well a classifier does using principal components rather than raw features.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
np.set_printoptions(suppress=True)

First we'll try with the raw features:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_data[features].to_numpy(),iris_data["species"].to_numpy(),test_size=0.33)

In [ ]:
#1-nn calssifier
pred = []
for test_feature in X_test:
  dist = [np.linalg.norm((test_feature - train_feature)) for train_feature in X_train]
  best_idx = np.argmin(dist)
  pred += [y_train[best_idx]]

(pred==y_test).sum()/y_test.size

np.float64(0.96)

Next, let's try with PCA, but instead of using all three principal components, let's just use the first (most important) component.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(components_df["PC1"].to_numpy(),iris_data["species"].to_numpy(),test_size=0.33)

Remember, our training data is just a single number for each entry (e.g., `0.36*sepal_length + 0.36*petal_width + 0.86*petal_length`):

In [ ]:
X_train #Only 1D features!

array([ 0.83825411,  0.65645231,  1.16790353, -2.83833761, -2.593687  ,
       -0.95798149, -2.27951233,  0.47333307,  2.326112  ,  0.63136914,
        0.62088242,  1.29022883, -2.9971111 , -2.57271356,  0.36325108,
        2.39203484,  2.30396199,  1.35351678,  3.77098732,  2.16865596,
        1.38864753, -2.53377136, -2.65609666,  0.1928143 ,  3.49120597,
       -2.5103039 ,  0.191936  ,  1.36312521, -2.9971111 ,  1.76567101,
        1.74733245, -2.60666774, -2.62008762,  1.93361378, -2.2904382 ,
        0.10605889,  2.44799816,  0.97443844, -2.63013519, -0.04061216,
       -2.15469301,  0.75369444,  0.32724204, -2.67912497,  1.47378722,
        2.63580715, -2.57021955,  2.55983674, -0.60105316,  1.95128426,
        0.93549624,  1.6797939 , -0.00284653,  0.87514145,  2.11922703,
       -0.12517183,  0.89904805, -2.37675446,  1.38864753,  1.94659451,
        0.90242036,  1.24079991,  2.62150897, -2.48434243, -2.74241292,
        0.11860048,  0.50816554,  2.83058968,  1.91483607,  1.25

How good of a classifier can we make knowing only a single number per entry?

In [ ]:
pred = []
for test_feature in X_test:
  dist = [np.linalg.norm((test_feature - train_feature)) for train_feature in X_train]
  best_idx = np.argmin(dist)
  pred += [y_train[best_idx]]

(pred==y_test).sum()/y_test.size

np.float64(0.88)

We achieve between 85-95% accuracy using only a 1D, single number feature!

Let'a ret kNN using the first 2 principal components

In [ ]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

def knn_classifier(k):
    def knn_classify(x_train, y_train, x_test):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(x_train, y_train)

        pred = knn.predict(x_test)
        return pred
    return knn_classify


def accuracy(classify, X_train, y_train, X_test, y_test):
  pred = classify(X_train, y_train, X_test)
  correct_pred = (pred == y_test)
  total_preds = y_test.size
  return correct_pred.sum()/total_preds

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(components_df[["PC1", "PC2"]].to_numpy(),iris_data["species"].to_numpy(),test_size=0.33)

In [ ]:
knn = knn_classifier(7)
accuracy(knn, X_train, y_train, X_test, y_test)

np.float64(0.94)

## Testing a new feature with PCA

Let's check out the data a final time to remind us of the raw features:

In [ ]:
iris_data[features+["species"]]

,sepal_length,petal_width,petal_length,species
0,5.1,0.2,1.4,setosa
1,4.9,0.2,1.4,setosa
2,4.7,0.2,1.3,setosa
3,4.6,0.2,1.5,setosa
4,5.0,0.2,1.4,setosa
...,...,...,...,...
145,6.7,2.3,5.2,virginica
146,6.3,1.9,5.0,virginica
147,6.5,2.0,5.2,virginica
148,6.2,2.3,5.4,virginica


We can try to classify a new data point not seen in training. In theory, we can do this manually by computing the equations for the new principal components from the base features. However, SKLearn has a helpful function for us: we can use `pca.transform` to transform this new data into the PCA space fit earlier.

In [ ]:
#Add a new flower with sepal_length = 5.2, petal_width=0.3, and petal_length=1.4
#This should be a "setosa", but let's see what KNN on just a 1D space says
new_data = pd.DataFrame([[5.2,0.3,1.4]], columns = ["sepal_length", "petal_width", "petal_length"])
new_data_pc = pca.transform(new_data)
print(new_data.to_numpy())
print(new_data_pc)

[[5.2 0.3 1.4]]
[[-2.58363943  0.30461016  0.12964552]]


In [ ]:
dists = [np.linalg.norm(new_data_pc[:,[0]] - train_feature) for train_feature in X_train[:,[0]]]
best_idx = np.argmin(dists)
print(y_train[best_idx])

setosa
